In [2]:
import pandas as pd
import os
from datasets import Dataset
from transformers import AutoTokenizer
import os
from utils import construct_prompt

current_dir = os.getcwd()
print(current_dir)
if current_dir.endswith("week_5_6"):
    os.chdir("../..")
    print(os.getcwd())
else:
    print("current dir", current_dir)


/opt/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/HansPeter/Dev/ku_local/nlp_course/src/nlp_course/code/week5
/Users/HansPeter/Dev/ku_local/nlp_course/src/nlp_course


In [2]:
ds_train = pd.read_parquet("dataset/train_df.parquet")
ds_val = pd.read_parquet("dataset/val_df.parquet")



In [ ]:


ds_train_inlang = ds_train[ds_train['answer_inlang'].notna()]
ds_val_inlang = ds_val[ds_val['answer_inlang'].notna()]
ds_train_en = ds_train
ds_val_en = ds_val


In [ ]:
from typing import Any, List, Optional
import json


import re
from datasets import Dataset    

def prompt_to_segments(
    prompt: str,
    tokenizer : AutoTokenizer,
) -> List[dict[str, Any]]:
    """
    Convert a prompt to a list of segments.
    """
    middle_text = 'assistant<\|end_header_id\|>'
    end_text = tokenizer.eos_token
    pattern = r'(.*?)assistant<\|end_header_id\|>(.*?)<\|eot_id\|>'
    match = re.search(pattern, prompt, re.DOTALL)
    chunks = [
        {"text": match.group(1), "label": "false"},
        {"text": middle_text, "label": "false"},
        {"text": match.group(2), "label": "true"},
        {"text": end_text, "label": "false"}
    ]
    return chunks

def construct_input_output_from_df(
    df: pd.DataFrame,
    tokenizer : AutoTokenizer,
    answer_key : str = "answer_inlang",
):
    segments = df.apply(
        lambda x: 
            prompt_to_segments(
                construct_prompt(
                    tokenizer, x['question'], x['context'], x[answer_key], tokenize=False
                ),
                tokenizer
            ), axis=1
        )

    data_dict = {'segments' : segments.tolist()}
    dataset = Dataset.from_pandas(pd.DataFrame.from_dict(data_dict))
    return dataset

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

train_jsonl = construct_input_output_from_df(
    ds_train_en, 
    tokenizer,
    "answer"
)

val_jsonl = construct_input_output_from_df(
    ds_val_en, 
    tokenizer,
    "answer"
)


train_jsonl.to_json(
    "code/week5/data/train_en_input_output.jsonl", 
    orient="records", 
    lines=True
)

val_jsonl.to_json(
    "code/week5/data/val_en_input_output.jsonl", 
    orient="records", 
    lines=True
)

train_jsonl.push_to_hub("hanspeterlyngsoeraaschoujensen/week41_train_en_input_output")
val_jsonl.push_to_hub("hanspeterlyngsoeraaschoujensen/week41_val_en_input_output")

